In [8]:
import sys
import os
from dotenv import load_dotenv

load_dotenv()

src_path = os.environ["src_path"] 
sys.path.append(src_path)

In [9]:
import pandas as pd
from src.paths import TRANSFORMED_DATA_DIR
from src.components.data_info import *
from dotenv import load_dotenv

In [ ]:
df = pd.read_parquet()

In [10]:
from datetime import datetime, timezone
from src.components.data_info import train_test_split

X_train, y_train, X_test, y_test = train_test_split(df, cutoff_date="", target_column_name = "target_demand_next_hour")

print(f"{X_train.shape=}")
print(f"{y_train.shape=}")
print(f"{X_test.shape=}")
print(f"{y_test.shape=}")

NameError: name 'df' is not defined

In [ ]:
X_train_no_date = X_train.drop(['date'], axis=1)

## Baseline model (Linear Regression)

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

model = LinearRegression()
model.fit(X_train_no_date, y_train)

In [ ]:
X_test_no_date = X_test.drop(['date'], axis=1)
y_pred = model.predict(X_test_no_date)
y_pred

In [ ]:
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import mean_absolute_error

def evaluate_model(y_test, y_pred):
    pass